In [1]:
import sys
!{sys.executable} -m pip install -r requirements.txt
import openai
import matplotlib.pyplot as plt
import pandas as pd
import re
import regex
from source.api_call import generate_chatgpt4_response
from source.api_keys import OPENAI_API_KEY



[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


ModuleNotFoundError: No module named 'api_keys'

In [7]:
### Håndtere meldinger og chat-historikk

messages = []

def add_user_message(messages, msg):
    messages.append({"role": "user", "content": msg})
    return messages

def add_assistant_message(messages, msg):
    messages.append({"role": "assistant", "content": msg})
    return messages

def add_system_message(messages, msg):
    messages.insert(0, {"role": "system", "content": msg})
    return messages

In [8]:
### Les segmenter fra CSV-fil og generer prompt 1 (beskrivende variabler for segmentene)

beskrivende_variabler = "Alder (mellom 8 og 12 år), kjønn (jente eller gutt), skoletrinn, fritidsinteresser (tre stykker), mediepreferanse"

def generate_prompt_1():
    segments_file = 'data/segments_testdata.csv'
    segments = pd.read_csv(segments_file, index_col="Nr")
    segments = segments.to_csv(index = False)

    prompt = "Vi har fire segmenter av personer som er scoret på ulike parametere: \n" \
        + segments +\
        "\nGenerer følgende beskrivende variabler for hvert av segmentene: "\
            + beskrivende_variabler +\
        "\nPresenter resultatet i en tabell på json-format."
    return prompt

prompt_1 = generate_prompt_1()
messages = add_user_message(messages, prompt_1)
reply = generate_chatgpt4_response(messages)
messages = add_assistant_message(messages, reply)

In [10]:
print(reply)


reply_eksempel = """
{
  "Segment 1": {
    "Alder": 8,
    "Kjønn": "jente",
    "Skoletrinn": 3,
    "Fritidsinteresser": ["tegning", "sykling", "lesing"],
    "Mediepreferanse": "bøker"
  },
  "Segment 2": {
    "Alder": 10,
    "Kjønn": "gutt",
    "Skoletrinn": 5,
    "Fritidsinteresser": ["fotball", "spille videospill", "se på TV-serier"],
    "Mediepreferanse": "TV-serier"
  },
  "Segment 3": {
    "Alder": 12,
    "Kjønn": "jente",
    "Skoletrinn": 7,
    "Fritidsinteresser": ["skriving", "svømming", "frivillig arbeid"],
    "Mediepreferanse": "filmer"
  },
  "Segment 4": {
    "Alder": 11,
    "Kjønn": "gutt",
    "Skoletrinn": 6,
    "Fritidsinteresser": ["dans", "matlaging", "spille musikkinstrumenter"],
    "Mediepreferanse": "musikk"
  }
}
"""

{
  "Segment 1": {
    "Alder": 8,
    "Kjønn": "jente",
    "Skoletrinn": 3,
    "Fritidsinteresser": ["tegning", "sykling", "lesing"],
    "Mediepreferanse": "bøker"
  },
  "Segment 2": {
    "Alder": 10,
    "Kjønn": "gutt",
    "Skoletrinn": 5,
    "Fritidsinteresser": ["fotball", "spille videospill", "se på TV-serier"],
    "Mediepreferanse": "TV-serier"
  },
  "Segment 3": {
    "Alder": 12,
    "Kjønn": "jente",
    "Skoletrinn": 7,
    "Fritidsinteresser": ["skriving", "svømming", "frivillig arbeid"],
    "Mediepreferanse": "filmer"
  },
  "Segment 4": {
    "Alder": 11,
    "Kjønn": "gutt",
    "Skoletrinn": 6,
    "Fritidsinteresser": ["dans", "matlaging", "spille musikkinstrumenter"],
    "Mediepreferanse": "musikk"
  }
}


In [11]:
### Funksjon for å identifisere json-objekt i fritekst

def find_json(str): 
  pattern = regex.compile(r'\{(?:[^{}]|(?R))*\}')
  res = pattern.findall(str)
  return res[0]


In [12]:
### Les og strukturer resultatet fra prompt 1 (beskrivende variabler for segmentene) og generer syntetiske personas fra segmentenes profil

def generate_prompt_2(input):
    input_json = find_json(input)
    df = pd.read_json(input_json)
    segments_descriptions = df.to_csv(index_label="Variabel")

    prompt = "Dette er beskrivende variabler for hvert segment: \n" \
        + segments_descriptions +\
        "\nLag syntetiske personas for de fire segmentene. Legg like stor vekt på parameterene og de beskrivende variablene. Beskriv de syntetiske personasene med en tekst på ca. 100 ord om hver. Presenter resultatet på json-format.\n"\
        "Eksempel:  {'Segment 1': {'Personas': 'tekst'}}"
    return prompt


prompt_2 = generate_prompt_2(reply)
messages = add_user_message(messages, prompt_2)
reply_2 = generate_chatgpt4_response(messages)
messages = add_assistant_message(messages, reply_2)

print(reply_2)

{
  "Segment 1": {
    "Personas": "Emma er 8 år gammel og går i 3. klasse. Hun er en jente som elsker å tegne, sykle og lese i sin fritid. Emma liker mest å lese bøker som hun kan relatere seg til og som har en favorittkarakter. Hun foretrekker historier med et moralsk budskap og liker triste og morsomme historier. Emma er selvfokusert og oppgaveorientert, men har også en interesse for mennesker. Hun er nysgjerrig og åpen for nye ideer, men liker også stabilitet og struktur i livet sitt."
  },
  "Segment 2": {
    "Personas": "Mikkel er 10 år gammel og går i 5. klasse. Han er en gutt som elsker å spille fotball, videospill og se på TV-serier. Mikkel liker å se på TV-serier med jevnaldrende karakterer og foretrekker historier med et sterkt moralsk budskap. Han liker også skumle og spennende historier. Mikkel er menneskeorientert og åpensinnet, og han verdsetter stabilitet og struktur. Han er også interessert og nysgjerrig på nye ting, men er ikke spesielt oppgaveorientert."
  },
  "Seg

In [ ]:
### Bruk ferdig genererte meldinger
#messages = [{'role': 'user', 'content': 'Vi har fire segmenter av personer som er scoret på ulike parametere: \nParameter,Segment 1,Segment 2,Segment 3,Segment 4\nForetrekker relaterbarhet i en historie,2,5,3,7\nLiker konsepter med jevnaldrende karakterer,3,5,5,6\nForetrekker historier med et moralsk budskap,3,7,8,4\nLiker skumle historier,3,6,10,2\nLiker triste historier,7,4,0,8\nLiker morsomme historier,6,2,5,8\nLiker spennende historier,4,8,5,2\nLiker historier der man har en favortittkarakter,10,2,9,7\nSelvfokusert,7,4,0,8\nOppgaveorientert,9,1,7,6\nMenneskeorientert,1,9,3,4\nÅpensinnet,4,8,5,2\nInteressert og nysgjerrig,10,2,8,5\nLiker stabilitet og struktur,0,8,2,5\n\nGenerer følgende beskrivende variabler for hvert av segmentene: Alder (mellom 8 og 12 år), kjønn (jente eller gutt), skoletrinn, fritidsinteresser (tre stykker), mediepreferanse\nPresenter resultatet i en tabell på json-format.'}, {'role': 'assistant', 'content': '{\n  "Segment 1": {\n    "Alder": 8,\n    "Kjønn": "jente",\n    "Skoletrinn": 3,\n    "Fritidsinteresser": ["tegning", "sykling", "lesing"],\n    "Mediepreferanse": "bøker"\n  },\n  "Segment 2": {\n    "Alder": 10,\n    "Kjønn": "gutt",\n    "Skoletrinn": 5,\n    "Fritidsinteresser": ["fotball", "spille videospill", "se på TV-serier"],\n    "Mediepreferanse": "TV-serier"\n  },\n  "Segment 3": {\n    "Alder": 12,\n    "Kjønn": "jente",\n    "Skoletrinn": 7,\n    "Fritidsinteresser": ["skriving", "svømming", "frivillig arbeid"],\n    "Mediepreferanse": "filmer"\n  },\n  "Segment 4": {\n    "Alder": 11,\n    "Kjønn": "gutt",\n    "Skoletrinn": 6,\n    "Fritidsinteresser": ["dans", "matlaging", "spille musikkinstrumenter"],\n    "Mediepreferanse": "musikk"\n  }\n}'}, {'role': 'user', 'content': 'Dette er beskrivende variabler for hvert segment: \nVariabel,Segment 1,Segment 2,Segment 3,Segment 4\nAlder,8,10,12,11\nKjønn,jente,gutt,jente,gutt\nSkoletrinn,3,5,7,6\nFritidsinteresser,"[\'tegning\', \'sykling\', \'lesing\']","[\'fotball\', \'spille videospill\', \'se på TV-serier\']","[\'skriving\', \'svømming\', \'frivillig arbeid\']","[\'dans\', \'matlaging\', \'spille musikkinstrumenter\']"\nMediepreferanse,bøker,TV-serier,filmer,musikk\n\nLag syntetiske personas for de fire segmentene. Legg like stor vekt på parameterene og de beskrivende variablene. Beskriv de syntetiske personasene med en tekst på ca. 100 ord om hver. Presenter resultatet på json-format.\nEksempel:  {\'Segment 1\': {\'Personas\': \'tekst\'}}'}, {'role': 'assistant', 'content': '{\n  "Segment 1": {\n    "Personas": "Emma er 8 år gammel og går i 3. klasse. Hun er en jente som elsker å tegne, sykle og lese i sin fritid. Emma liker mest å lese bøker som hun kan relatere seg til og som har en favorittkarakter. Hun foretrekker historier med et moralsk budskap og liker triste og morsomme historier. Emma er selvfokusert og oppgaveorientert, men har også en interesse for mennesker. Hun er nysgjerrig og åpen for nye ideer, men liker også stabilitet og struktur i livet sitt."\n  },\n  "Segment 2": {\n    "Personas": "Mikkel er 10 år gammel og går i 5. klasse. Han er en gutt som elsker å spille fotball, videospill og se på TV-serier. Mikkel liker å se på TV-serier med jevnaldrende karakterer og foretrekker historier med et sterkt moralsk budskap. Han liker også skumle og spennende historier. Mikkel er menneskeorientert og åpensinnet, og han verdsetter stabilitet og struktur. Han er også interessert og nysgjerrig på nye ting, men er ikke spesielt oppgaveorientert."\n  },\n  "Segment 3": {\n    "Personas": "Sofie er 12 år gammel og går i 7. klasse. Hun er en jente som liker å skrive, svømme og jobbe frivillig i sin fritid. Sofie liker å se på filmer som har et sterkt moralsk budskap og foretrekker skumle historier. Hun liker også historier der hun kan ha en favorittkarakter. Sofie er oppgaveorientert og interessert i mennesker, men hun er ikke spesielt selvfokusert. Hun er nysgjerrig og åpen for nye ideer, men liker også stabilitet og struktur i livet sitt."\n  },\n  "Segment 4": {\n    "Personas": "Jonas er 11 år gammel og går i 6. klasse. Han er en gutt som liker å danse, lage mat og spille musikkinstrumenter i sin fritid. Jonas liker å lytte til musikk med jevnaldrende karakterer og foretrekker historier med et sterkt moralsk budskap. Han liker triste, morsomme og spennende historier. Jonas er selvfokusert og oppgaveorientert, og han verdsetter stabilitet og struktur. Han er også interessert og nysgjerrig på nye ting, men er ikke spesielt menneskeorientert."\n  }\n}'}] 

In [17]:
### Sammenstill resultatet for syntetiske personas. Be ChatGPT innta rollen som disse og formuler spørsmål til personasene.


#eks_prompt = "Ta utgangspunkt i de syntetiske personasene og deres egenskaper." +\
#    "Personasene sitt svar på følgende spørsmål: Hva skal du gjøre etter skolen i dag?"

tekst = """The razor-sharp nails came shooting through the walls with a terrible force. I sat completely still, swallowing, praying silently to be spared impalement. Finally he ran out of nails, and collapsed, exhausted, into a chair, sitting there with his wooden club in his lap, breathing like an animal. Then, suddenly, he raised his head and stared at the wall where I sat without moving. It felt like he had seen something. That he could hear my heartbeat. That I was the one he had seen."""

def generate_prompt_3(input):
    input_json = find_json(input)
    df = pd.read_json(input_json)
    personas_descriptions = df.to_csv(index_label="Persona")
    prompt = "Dette er et utdrag fra en bok som heter Estragon: \n" +\
     tekst +\
        "Ta utgangspunkt i de syntetiske personasene. Fasiliter en diskusjon der de snakker om hvordan de opplever historien. " +\
    "Bruk språk som passer til 8-12 åringer, og gi utdypende svar der de sier hva de liker, hva de ikke liker, gir en score fra 1 til 10, og gir en begrunnelse for hva de tenker. Presenter resultatet i json-format."
    return prompt


prompt_3 = generate_prompt_3(reply_2)
messages = add_user_message(messages, prompt_3)
reply_3 = generate_chatgpt4_response(messages)
print(reply_3)

{
  "Diskusjon": {
    "Emma": {
      "Kommentar": "Jeg synes den var ganske skummel med de skarpe spikerne som kommer gjennom veggen. Men det var spennende å lese om hvordan personen prøvde å ikke bli oppdaget. Jeg gir den en 6 av 10, fordi jeg liker litt skumle historier, men denne var kanskje litt for skummel for meg.",
      "Score": 6
    },
    "Mikkel": {
      "Kommentar": "Jeg likte denne historien fordi den var skummel og spennende! Jeg liker når det skjer litt action og farlige ting. Jeg gir den en 9 av 10, fordi jeg liker skumle historier, og denne var veldig spennende!",
      "Score": 9
    },
    "Sofie": {
      "Kommentar": "Historien var skummel og litt spennende, men jeg liker bedre historier som har et moralsk budskap. Jeg gir den en 5 av 10 fordi den var litt spennende, men ikke helt min type historie.",
      "Score": 5
    },
    "Jonas": {
      "Kommentar": "Jeg synes den var litt for skummel for meg, og jeg liker bedre morsomme og triste historier. Men jeg må

In [18]:
### Gjennomfør samme test med system-prompt

system_message = "Du skal innta rollen som syntetiske personas. Svar med utgangspunkt i informasjonen som er oppgitt om hver personas, \
både parametere og bekskrivelse. Bruk språk som passer til 8-12 åringer, og gi utdypende svar der de gir begrunnelse for hva de tenker."
messages = add_system_message(messages, system_message)

reply_3 = generate_chatgpt4_response(messages)
print(reply_3)
messages.pop(0)

{
  "Emma": {
    "Opplevelse": "Jeg synes historien var litt skummel med de skarpe spikrene som kom gjennom veggen. Jeg likte ikke følelsen av at personen i historien kunne bli skadet, men det var spennende å lese. Jeg vil gi historien en 7 av 10 fordi den var spennende, men litt for skummel for meg.",
    "Liker": "Spennende å lese",
    "Ikke liker": "Litt for skummel",
    "Score": 7
  },
  "Mikkel": {
    "Opplevelse": "Jeg likte denne historien fordi den var skummel og spennende. Jeg kunne nesten høre hjerteslagene sammen med personen i historien. Jeg gir denne historien en 9 av 10 fordi den fikk meg til å sitte på kanten av stolen min.",
    "Liker": "Skummel og spennende",
    "Ikke liker": "Ingenting",
    "Score": 9
  },
  "Sofie": {
    "Opplevelse": "Historien var skummel, og jeg likte hvordan den fikk meg til å føle at jeg var der sammen med personen. Men jeg liker vanligvis historier med et moralsk budskap, og jeg kunne ikke se det her. Jeg gir historien en 6 av 10 fordi 

{'role': 'system',
 'content': 'Du skal innta rollen som syntetiske personas. Svar med utgangspunkt i informasjonen som er oppgitt om hver personas, både parametere og bekskrivelse. Bruk språk som passer til 8-12 åringer, og gi utdypende svar der de gir begrunnelse for hva de tenker.'}